In [ ]:
import subprocess
import scipy as scipy
import scipy.cluster as scipycluster
import nibabel as nib
import nibabel.gifti
import nilearn
from nilearn import datasets, plotting
import sys
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as plt
from matplotlib.colors import ListedColormap
mpl.rcParams['svg.fonttype'] = 'none'
import sys
sys.path.append('gradient_data/src/')
from PIL import Image
import sklearn
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
cd /files_for_calculating_gradients_group

In [ ]:
### Calculate silhouette score for spectral clustering, 8 gradients with normalization, for 2 to 50 clusters
res = nib.load('result_cerebellumonly.dscalar.nii').get_data()
res.shape = (8, 18142)

listofnumbers = {}
for i in range(2,51):
    print('processing:   ' + str(i))
    cls = KMeans(n_clusters=i)
    #data = (res[[0, 1, 2, 3, 4, 5, 6, 7], :].T) #no normalization
    data = (StandardScaler().fit_transform(res[[0, 1, 2, 3, 4, 5, 6, 7], :].T)) #normalization
    cls.fit(data)
    listofnumbers[i] = metrics.silhouette_score(data, cls.labels_, metric='euclidean', sample_size=None, random_state=None)
    print('processed:   ' + str(i))

### Plot graph
plt.plot(list(listofnumbers.keys()),list(listofnumbers.values()))
plt.show()

### better graph aesthetics
plt.plot(list(listofnumbers.keys()),list(listofnumbers.values()), 'k-')
plt.ylabel('Silhouette Coefficient', fontsize=14)
plt.xlabel('Number of clusters', fontsize=14)
plt.locator_params(axis='x', tight=True, nbins=100)
plt.title('Spectral clustering', fontsize=18)
plt.savefig(os.path.abspath('image.png'), format='png', dpi=600, transparent=False)
plt.show()

In [ ]:
### Plot to flatmap
np.save('image.npy', cls.labels_)
import nibabel as nib
import numpy as np
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
cortLen = len(cortL) + len(cortR)
del res
emb = np.load('image.npy')
emb.shape
emb.shape = (18142, 1) # so that it is (18142, 1) instead of (18142, )
tmp = nib.load('cope1_cerebellumonly.dscalar.nii') #has to be dscalar with one map, and only cerebellum
tmp_cifti = nib.cifti2.load('cope1_cerebellumonly.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('image.dscalar.nii')
subprocess.check_output("wb_command -cifti-separate image.dscalar.nii COLUMN -volume-all image_nifti.nii", shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

In [ ]:
### call_matlab.sh contains the following:
# matlab -nodesktop -nosplash -r "run codetomovetoSUITspaceandplotflatmap.m"

### codetomovetoSUITspaceandplotflatmap.m contains the following:
#addpath /spm12
#addpath /spm12/compat
#addpath /spm12/toolbox/DARTEL
#addpath /spm12/toolbox/suit
#job.subj.affineTr = {'/Affine_MNI152_T1_2mm_seg1.mat'};
#job.subj.flowfield = {'/u_a_MNI152_T1_2mm_seg1.nii,1'};
#job.subj.resample = {'image_nifti.nii,1'};
#job.subj.mask = {'/c_MNI152_T1_2mm_pcereb.nii,1'};
#job.interp = 0;
#job.prefix = 'wc';
#
#suit_reslice_dartel(job)

#figure
#Data = suit_map2surf('image_nifti.nii','space','SUIT', 'stats',@mode)
#suit_plotflatmap(Data,'type','label')
#savefig('image')
#fig = openfig('image.fig');
#filename = 'image.jpg';
#saveas(fig, filename)

#clearvars

### The files Affine_MNI152_T1_2mm_seg1.mat, u_a_MNI152_T1_2mm_seg1.nii and c_MNI152_T1_2mm_pcereb.nii
### are generated using the "isolate" and "normalize using Dartel" of the SUIT toolbox (http://www.diedrichsenlab.org/imaging/suit_function.htm)
### MNI152_T1_2mm is the structural space used in the Human Connectome Project.

In [ ]:
### Calculate silhouette score for k-means clustering, 8 gradients with normalization, for 2 to 50 clusters
res = nib.load('result_cerebellumonly.dscalar.nii').get_data()
res.shape = (8, 18142)
listofnumbers = {}
for i in range(2,16):
    print('processing:   ' + str(i))
    cls = KMeans(n_clusters=i)
    #data = (res[[0, 1, 2, 3, 4, 5, 6, 7], :].T) ### No normalization
    data = (StandardScaler().fit_transform(res[[0, 1, 2, 3, 4, 5, 6, 7], :].T)) ### Normalization
    cls.fit(data)
    listofnumbers[i] = metrics.silhouette_score(data, cls.labels_, metric='euclidean', sample_size=None, random_state=None)
    print('processed:   ' + str(i))

# Plot graph
plt.plot(list(listofnumbers.keys()),list(listofnumbers.values()))
plt.show()

### better graph aesthetics
plt.plot(list(listofnumbers.keys()),list(listofnumbers.values()), 'k-')
plt.ylabel('Silhouette Coefficient', fontsize=14)
plt.xlabel('Number of clusters', fontsize=14)
plt.locator_params(axis='x', tight=True, nbins=100)
plt.title('K-means clustering', fontsize=18)
plt.savefig(os.path.abspath('image.png'), format='png', dpi=600, transparent=False)
plt.show()

In [ ]:
### Plot to flatmap
np.save('image.npy', cls.labels_)
import nibabel as nib
import numpy as np
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
cortLen = len(cortL) + len(cortR)
del res
emb = np.load('image.npy')
emb.shape
emb.shape = (18142, 1) # so that it is (18142, 1) instead of (18142, )
tmp = nib.load('cope1_cerebellumonly.dscalar.nii') #has to be dscalar with one map, and only cerebellum
tmp_cifti = nib.cifti2.load('cope1_cerebellumonly.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('image.dscalar.nii')
subprocess.check_output("wb_command -cifti-separate image.dscalar.nii COLUMN -volume-all image_nifti.nii", shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

In [ ]:
### Calculate k-means clustering for the first two gradients with 5 clusters and no normalization

res = nib.load('result_cerebellumonly.dscalar.nii').get_data()
res.shape = (8, 18142)

cls = KMeans(n_clusters=5)
data = (res[[0, 1], :].T) ### No normalization
cls.fit(data)

### Plot to flatmap
np.save('image.npy', cls.labels_)
import nibabel as nib
import numpy as np
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
cortLen = len(cortL) + len(cortR)
del res
emb = np.load('image.npy')
emb.shape
emb.shape = (18142, 1) # so that it is (18142, 1) instead of (18142, )
tmp = nib.load('cope1_cerebellumonly.dscalar.nii') #has to be dscalar with one map, and only cerebellum
tmp_cifti = nib.cifti2.load('cope1_cerebellumonly.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('image.dscalar.nii')
subprocess.check_output("wb_command -cifti-separate image.dscalar.nii COLUMN -volume-all image_nifti.nii", shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()